In [1]:
import pyspark.sql.functions as func
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, DataFrame
import pyarrow.parquet as pq
import pandas as pd
import os

In [2]:
path_parquet = '../src/test/resources/flattening/parquet-table/single_table'
path_PMSI_flat = '../src/test/resources/flattening/parquet-table/flat_table/PMSI_Flat'

In [3]:
os.listdir(path_parquet + '/MCO_D')

['year=2007', 'year=2008', 'year=2006']

In [4]:
 pd.read_parquet(path_parquet + '/MCO_D')

,ASS_DGN,ETA_NUM,RSA_NUM,RSS_NUM,year
0,C66.9,10000123,10000543,217,2006
1,C66.5,10000123,20000123,217,2007
2,C652,10000123,30000852,217,2008


In [5]:
liste_bases = [base for base in os.listdir(path_parquet) if ("MCO" in base)]

In [6]:
liste_bases

['MCO_D', 'MCO_C', 'MCO_B', 'MCO_UM', 'MCO_A']

In [7]:
bases_MCO = {}

for base in liste_bases:
    path = path_parquet + '/' + base
    bases_MCO[base] = pd.DataFrame()
    for year in os.listdir(path):
        temp_df = pd.read_parquet(path = path + '/' + year, engine="pyarrow")
        temp_df['year'] = int(year.split("=")[1])
        bases_MCO[base] = pd.concat([bases_MCO[base], temp_df])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [8]:
bases_MCO['MCO_C']

,DAT_RET,ENT_DAT,ETA_NUM,FHO_RET,FOR_NUM,NAI_RET,NIR_RET,NUM_ENQ,PMS_RET,RSA_NUM,SEJ_NUM,SEJ_RET,SEX_RET,SOR_ANN,SOR_DAT,SOR_MOI,VID_HOSP_FOR,year
0,0,NaN,10000123,0,NaN,0,0,Patient_02,0,20000123,50,0,0,2007,NaN,2,NaN,2007
1,0,NaN,10000123,0,NaN,0,0,Patient_02,0,20000345,55,0,0,2007,NaN,2,NaN,2007
0,0,08032008,10000123,0,2,0,0,Patient_02,0,30000546,90,0,0,2008,10032008,3,10,2008
1,0,15032008,10000123,0,2,0,0,Patient_02,0,30000852,95,0,0,2008,17032008,3,10,2008
0,0,NaN,10000123,0,NaN,0,0,Patient_02,0,10000987,4,0,0,2006,NaN,1,NaN,2006
1,0,NaN,10000123,0,NaN,0,0,Patient_02,0,10000543,9,0,0,2006,NaN,1,NaN,2006


In [9]:
for base in bases_MCO:
    if base != 'MCO_B':
        new_cols = [base + '__'+ col if col not in ['ETA_NUM', 'RSA_NUM', 'year'] else col for
                    col in bases_MCO[base].columns ]
        bases_MCO[base].columns = new_cols
    print(base,' : ',bases_MCO[base].shape, " ", bases_MCO[base].columns)

MCO_D  :  (3, 5)   Index(['MCO_D__ASS_DGN', 'ETA_NUM', 'RSA_NUM', 'MCO_D__RSS_NUM', 'year'], dtype='object')
MCO_C  :  (6, 18)   Index(['MCO_C__DAT_RET', 'MCO_C__ENT_DAT', 'ETA_NUM', 'MCO_C__FHO_RET',
       'MCO_C__FOR_NUM', 'MCO_C__NAI_RET', 'MCO_C__NIR_RET', 'MCO_C__NUM_ENQ',
       'MCO_C__PMS_RET', 'RSA_NUM', 'MCO_C__SEJ_NUM', 'MCO_C__SEJ_RET',
       'MCO_C__SEX_RET', 'MCO_C__SOR_ANN', 'MCO_C__SOR_DAT', 'MCO_C__SOR_MOI',
       'MCO_C__VID_HOSP_FOR', 'year'],
      dtype='object')
MCO_B  :  (6, 64)   Index(['AGE_ANN', 'AGE_GES', 'AGE_JOU', 'BDI_COD', 'BDI_DEP', 'BEB_SEJ',
       'BEH_NBJ', 'CAI_SUP_NBR', 'COD_IGS', 'COD_SEX', 'DGN_PAL', 'DGN_REL',
       'DLY_ACT', 'ENT_MOD', 'ENT_PRV', 'ETA_NUM', 'ETE_GHS_NUM',
       'GHM_24705Z_ACT', 'GHM_24706Z_ACT', 'GHM_24707Z_ACT', 'GHS_9510_ACT',
       'GHS_9511_ACT', 'GHS_9512_ACT', 'GHS_9515_ACT', 'GHS_9524_ACT',
       'GHS_NUM', 'GRC_GHM', 'GRC_RET', 'GRC_VER', 'GRG_GHM', 'GRG_RET',
       'GRG_VER', 'NBR_ACT', 'NBR_DGN', 'NBR_RUM', 

# Construction de la base applatie

MCO_B / MCO_C

In [10]:
table_centrale = pd.merge(bases_MCO['MCO_B'], bases_MCO['MCO_C'], on=['ETA_NUM', 'RSA_NUM'], how='inner')
print(table_centrale.shape)
table_centrale.to_parquet(path_PMSI_flat + '/table_centrale.parquet', compression=None) 

(6, 80)


ajout de MCO_A

In [11]:
temp_MCO_A = pd.merge(table_centrale, bases_MCO['MCO_A'], on=['ETA_NUM', 'RSA_NUM'], how='inner')
temp_MCO_A.to_parquet(path_PMSI_flat + '/tc_MCO_A.parquet', compression=None) 
print(temp_MCO_A.shape)
PMSI_flat = temp_MCO_A

(6, 92)


ajout de MCO_D

In [12]:
temp_MCO_D = pd.merge(table_centrale, bases_MCO['MCO_D'], on=['ETA_NUM', 'RSA_NUM'], how='inner')
temp_MCO_D.to_parquet(path_PMSI_flat + '/tc_MCO_D.parquet', compression=None) 
print(temp_MCO_D.shape)
PMSI_flat = pd.concat([PMSI_flat, temp_MCO_D], sort=True)
print(PMSI_flat.shape)

(3, 83)
(9, 94)


ajout de MCO_UM

In [13]:
temp_MCO_UM = pd.merge(table_centrale, bases_MCO['MCO_UM'], on=['ETA_NUM', 'RSA_NUM'], how='inner')
temp_MCO_UM.to_parquet(path_PMSI_flat + '/tc_MCO_UM.parquet', compression=None) 
print(temp_MCO_UM.shape)
PMSI_flat = pd.concat([PMSI_flat, temp_MCO_UM], sort=True)
print(PMSI_flat.shape)
del PMSI_flat['year_x'], PMSI_flat['year_y'] 
PMSI_flat.to_parquet(path_PMSI_flat + '/PMSI_flat.parquet', compression=None, index=False) 

(2, 84)
(11, 97)


In [15]:
PMSI_flat.shape

(11, 95)